In [1]:
from LP import simulate_LP, LP_sol
from KG import simulate_KG, KG_score, opt_KG_score
from multiprocessing import Pool

In [2]:
# return [N, M, mean, std, computation time]
def simulation(T, alpha, N, M, policy):
    if policy == "fp":
        return simulate_LP(T, alpha, N, M)
    if policy == "kg":
        return simulate_KG(T, alpha, N, M, KG_score)
    if policy == "okg":
        return simulate_KG(T, alpha, N, M, opt_KG_score)
    raise Exception(f"Policy {policy} not recognized.")

def wrapper(args):
    return simulation(*args)

In [3]:
# return [fp results, kg results, okg results]
# result = [optimality gap, T, alpha, number of arm, number of rounds]
def simulation_all_policies(T, alpha, N, M):
    m = LP_sol(T=T, alpha=alpha)
    objVal = m.objVal
    
    args = (T, alpha, N, M)
    with Pool(3) as p:
        res = p.map(wrapper, [(*args, "fp"), (*args, "kg"), (*args, "okg")])
    res = [[objVal * N - item[2]] + list(item) for item in res]
    return res

In [4]:
import pandas as pd
import numpy as np

In [5]:
fp_df  = pd.DataFrame(index=["opt-gap", "N", "M", "expect-reward", "std", "time"])
kg_df  = pd.DataFrame(index=["opt-gap", "N", "M", "expect-reward", "std", "time"])
okg_df = pd.DataFrame(index=["opt-gap", "N", "M", "expect-reward", "std", "time"])

In [6]:
T, alpha = 7, 1/4

In [15]:
for N in range(40, 160, 40):
    res = simulation_all_policies(T, alpha, N, 10 * N)
    dfs = [fp_df, kg_df, okg_df]
    fnames = ["fp.csv", "kg.csv", "okg.csv"]
    for i in range(3):
        dfs[i][N] = res[i]
        dfs[i].to_csv(fnames[i], index=True)
    print(f"N={N} finished.")
    
    
